# Домашнее задание № 10. Генерация текста

### Задание 1 (8 баллов).

Попробуйте дообучать GPT на каком-то другом тексте (можете попробовать любые стихи или какие-то специфичные вещи вроде анекдотов, теорий заговоров, постов в помоечных телеграм каналах, текстов журналистов и СМИ с выразительным стилем). 
Попробуйте разные методы и параметры генерации (beam search, температура, top_k и тп). Сохраните в тетрадке несколько хороших сгенерированных текстов


In [1]:
#!pip install transformers

In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
DEVICE = torch.device("cuda:0")

# возьмем модель поменьше, так как дообучение это обновление весов
model_name_or_path = "sberbank-ai/rugpt3small_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path, use_cache=False).to(DEVICE)

In [3]:
from transformers import TextDataset, DataCollatorForLanguageModeling

train_path = '/kaggle/input/toxic-comments/some_text.txt'

In [4]:
with open(train_path, 'r') as f: #open the file
    contents = f.readlines() #put the lines to a variable (list).

# Вид данных

In [15]:
contents[:5]

['скотина! что сказать\n',
 'я сегодня проезжала по рабочей и между домами снитенко и гомолысовой магазином ( на пустыре) бежала кошка похожего окраса. может, я и ошиблась, но необычный окрас бросился в глаза.\n',
 'очередной лохотрон. зачем придумывать очередной налог на воздух, если можно обьявить инсульт и грипп- пандемией! и лихо на придурках зарабатывать годами на штрафах, фейковых вакцинах, всевозможных платных тестах, продажей масок и перчаток по баснословным ценам.. самое смешное, что бараны блеют и верят пастуху, телевизору. живут как под гипнозом. не думая, не глядя по сторонам.\n',
 'ретро дежавю ... сложно понять чужое сердце , лиш ощутить музыкой видимо\n',
 'а когда мы статус агрогородка получили?\n']

# Кол-во данных

In [36]:
len(contents)

10000

In [5]:
path_datatrain = '/kaggle/working/toxic_comments.txt' # для каггла

In [6]:
with open(path_datatrain, "w", encoding="utf-8") as traindata:
    for sentence in contents:
        traindata.writelines(sentence)

In [7]:
# Создание датасета
train_dataset = TextDataset(tokenizer=tokenizer,file_path=path_datatrain,block_size=64, 
                            overwrite_cache=True)

# специальный класс который будет подавать в модель данные в нужном ей виде
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

/opt/conda/lib/python3.7/site-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [8]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments( 
    output_dir= "./finetuned",
    overwrite_output_dir=True,
    num_train_epochs=100, 
    per_device_train_batch_size=64, 
    per_device_eval_batch_size=64,  
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5),None))

In [9]:
trainer.train()

***** Running training *****
  Num examples = 3390
  Num Epochs = 100
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 2700
  Number of trainable parameters = 125231616
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
500,4.318900
1000,3.818800
1500,3.507200
2000,3.300200
2500,3.182800


Saving model checkpoint to ./finetuned/checkpoint-500
Configuration saved in ./finetuned/checkpoint-500/config.json
Configuration saved in ./finetuned/checkpoint-500/generation_config.json
Model weights saved in ./finetuned/checkpoint-500/pytorch_model.bin
/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Saving model checkpoint to ./finetuned/checkpoint-1000
Configuration saved in ./finetuned/checkpoint-1000/config.json
Configuration saved in ./finetuned/checkpoint-1000/generation_config.json
Model weights saved in ./finetuned/checkpoint-1000/pytorch_model.bin
/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  

TrainOutput(global_step=2700, training_loss=3.5898939344618057, metrics={'train_runtime': 3525.3092, 'train_samples_per_second': 96.162, 'train_steps_per_second': 0.766, 'total_flos': 1.1072249856e+16, 'train_loss': 3.5898939344618057, 'epoch': 100.0})

TrainOutput(global_step=2700, training_loss=3.5898939344618057, metrics={'train_runtime': 3525.3092, 'train_samples_per_second': 96.162, 'train_steps_per_second': 0.766, 'total_flos': 1.1072249856e+16, 'train_loss': 3.5898939344618057, 'epoch': 100.0})

In [10]:
text = "Мой друг написал книгу"
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        temperature=0.9,
                        top_k=30,
                        max_length=60,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1",
  "use_cache": false
}

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Мой друг написал книгу про ментов
постепенно, не сразу, конечно. постепенно, но накапливается.
а ты уже в списке врагов россии.
приветик) не помешал бы))
с днем рождения, дорогая! хорошего отдыха, пусть твой малыш растет счастливым, пусть у


In [14]:
text = "Вчера я смотрел телевизор и"
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        temperature=1.4,
                        top_k=50,
                        max_length=100,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1",
  "use_cache": false
}

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Вчера я смотрел телевизор и обратил на себя внимание,как будто мои глаза разулись и начали что-то высматривать по сторонам......а в комнате стоял гул от грохота...это случилось, когда я стоял на балконе, в кухне и вдруг я почувствовал дул прямо в лоб стоявшей около мне в коридоре няне.
а мне уже в 20лет надоели эти диеты с негативом
у меня тоже самое. в саду яблок и груш много, я каждую складываю в баночку


### Получается плохо. Уменьшим температуру и top_k (случайность).

In [16]:
text = "Вчера я смотрел телевизор и"
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                         do_sample=True,
                         temperature=0.2,
                         top_k=20,
                         max_length=50,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1",
  "use_cache": false
}

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Вчера я смотрел телевизор и думал, что же я такого не сделал? ведь я же не виноват, что у меня нет машины, и я не гулял на улице. я просто не знал, что мне делать. ведь я же не виноват, что


А так происходит зацикливание на одних и тех же словах.

In [26]:
text = "Вчера я смотрел телевизор и"
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        temperature=1.4,
                        top_k=20,
                        max_length=100,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1",
  "use_cache": false
}

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Вчера я смотрел телевизор и вспомнил один сюжет который мне задала одна моя знакомая- она приезжала на лето к подруге отдыхать к ней а там одна из них пьяная на траулеере каталась по каким то улицам на белой машине. это точно она! у нее в тот вечер было не хорошо настроение и она пригласила на ужин знакомого она пришла и выпила с ним немного..а в её телефоне в этот момент не было номера её мужа, она видимо его запомнила... вот такая интересная судьба...�


Получилась короткая странная история. Глупая, но относительно связная.

In [29]:
text = "У меня отец работает "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        temperature=1.4,
                        top_k=20,
                        max_length=100,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1",
  "use_cache": false
}

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



У меня отец работает!! я его очень уважаю!!!
вы просто класс☺️️ 👍🤪
это не просто красиво и красиво, это великолепно!
везде люди работают,а не болтаются за счёт простых людей!
очень хорошо вы написали, что у них есть мозги...))) а вы их где берете????
это не развод а провокация(tr) (tr)
напишите в техподдержку если


Тут уже похуже. Но становится очевидно, что данные, наверное, не очень подходят под задачу -- изначально комментарии довольно "бессвязны" и модель им и подражает.

Num Beams оперирует смайликами


In [35]:
text = "У меня отец работает "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=False, num_beams=5,
                        max_length=100,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)
num_beams=10,

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1",
  "use_cache": false
}

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



У меня отец работает 💋💋💋💋💋💋💋💋💋💋💋💋💋💋💋💋💋💋💋💋💋💋💋💋💋💋💋💋💋💋💋�


In [34]:
text = "Каждый день я "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=False, num_beams=15,
                        max_length=50,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)
num_beams=10,

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1",
  "use_cache": false
}

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Каждый день я 🤦🤦🤦🤦🤦🤦🤦🤦🤦🤦🤦🤦🤦🤦🤦�


### Задание  2 (2 балла)

Ответьте на следующие вопросы:

1) В каких статья были представлены GPT-1, GPT-2, GPT-3?

2) Как собирался обучающий корпус для GPT-3? Каким образом создатели старались обеспечить высокое качество текстов в обучающей выборке?

1. GPT-1 paper Improving Language Understanding by Generative Pre-training.

2. GPT-2 paper Language Models are unsupervised multitask learners.

3. GPT-3 paper Language models are few shot learners.

Кратко:

GPT-3 был обучен на пяти различных корпусах, каждому из которых был присвоен определенный вес. Наборы данных высокого качества отбирались чаще, и модель обучалась на них в течение более чем одной эпохи. Использовались датасеты: Common Crawl, WebText, Books1, Books2 и Wikipedia.

Подробнее: 

Чтобы улучшить среднее качество наборов данных, разработчики сделали следующее:

* отфильтровали версию CommonCrawl на основе сходства с рядом высококачественных эталонных корпусов;
* выполнили нечеткую дедубликацию на уровне документа (внутри) и между наборами данных, чтобы предотвратить избыточность и сохранить целостность валидационного набора данных, применяемого для оценки переобучения;
* для разнообразия добавили известные высококачественные эталонные корпуса в набор тренировочных данных: WebText, содержащий тексты с сайтов, страницы которых были отмечены пользователями, как полезные по контенту; корпуса книг Books1 и Books2; англоязычную Википедию.

Во время обучения наборы данных отбирались не пропорционально их размеру. Более качественные наборы выбирались чаще, так что наборы данных Common Crawl и Books2 выбирались менее одного раза во время обучения, а другие наборы данных отбирались по 2–3 раза.